### Provision Layer for American Express, credit account

**Author**: Ricardo Pérez Castillo

**Latest update**: 2024-12-30

**Version**: 6.0

**Purpose**: Prepare expense data into an unified single source of truth.

### Table of Contents
1. [Introduction](#introduction)
2. [Raw File Importing](#raw-file-importing)
3. [Data Description](#data-description)
4. [Basic Data Cleansing](#basic-data-cleansing)
5. [Entity Harmonization](#entity-harmonization)
6. [Transaction Type Definition](#transaction-type-definition)
7. [Data Cleansing and Transformation](#data-cleansing-and-transformation)
8. [Exporting](#exporting)

### Introduction

American Express is a credit provider. When extracting data from their portal, select the option that provides the most detail possible. Extract in excel format.

### Raw File Importing

In [1]:
# Import necessary libraries
import pandas as pd  # For data manipulation and analysis
import numpy as np  # For numerical computations
from datetime import datetime  # To handle date and time operations
import pathlib  # For handling and manipulating file paths in an object-oriented way
from dateutil.relativedelta import relativedelta  # To handle date arithmetic

# Import custom configurations and mappings
from config import current_month, current_month_text, current_year  # Custom configurations for date handling
from entity_mapping import entity_mapping  # Predefined mapping for harmonizing entities
from transaction_subtype_mapping import transaction_subtype_mapping  # Predefined mapping for harmonizing transaction subtypes

# Import utilities for fuzzy matching
from difflib import get_close_matches  # To find close matches between strings

# Import utilities for generating hash values
import hashlib  # For generating hash values

# Import utilities to remove spanish accents
import unicodedata # For removing accents from strings

In [2]:
# Define the base directory for file storage
base_dir = pathlib.Path.home() / "Documents" / "Finanzas"
# The pathlib.Path.home() dynamically retrieves the user's home directory.

# Define the input directory for files
input_dir = base_dir / "AMEX" / "Movimientos" / "2024"
# This constructs the full path to the directory where input files are stored.
# Example: "C:/Users/YourUserName/Documents/Finanzas/BBVA/BBVA TDC/Movimientos/2024"

# Define the output directory for processed files
output_dir = base_dir / "Personal Spend" / "02 Individual Datasets" / "2024"
# This constructs the full path to the directory where processed output files will be saved.
# Example: "C:/Users/YourUserName/Documents/Finanzas/Personal Spend/02 Individual Datasets/2024"

In [3]:
# Function to construct the input file path for a specific month and year
def get_filename_input(month, year, month_text, prefix="", suffix="", extension=".xlsx"):
    """
    Constructs a filename for the input file based on the provided parameters.

    Parameters:
    - month (int): The numeric month (e.g., 1 for January, 12 for December).
    - year (int): The year (e.g., 2024).
    - month_text (str): The textual representation of the month (e.g., "January").
    - prefix (str): An optional prefix for the file name (default is an empty string).
    - suffix (str): An optional suffix for the file name (default is an empty string).
    - extension (str): The file extension (default is ".xlsx").

    Returns:
    - str: The constructed file name (e.g., "01_suffix.xlsx").
    """
    return str(month).zfill(2) + suffix + extension


# Function to construct the output file path for a specific month and year
def get_filename_output(month, year, month_text, prefix="", suffix="", extension=".csv"):
    """
    Constructs a filename for the output file based on the provided parameters.

    Parameters:
    - month (int): The numeric month (e.g., 1 for January, 12 for December).
    - year (int): The year (e.g., 2024).
    - month_text (str): The textual representation of the month (e.g., "January").
    - prefix (str): An optional prefix for the file name (default is an empty string).
    - suffix (str): An optional suffix for the file name (default is an empty string).
    - extension (str): The file extension (default is ".csv").

    Returns:
    - str: The constructed file name with the format "month_text/month_textMM.csv".
    """
    return f"{month_text}/{prefix}{month_text}{str(month).zfill(2)}{extension}"


In [ ]:
# Create file paths for the current month's input and output files

# Construct the full input file path
input_file = input_dir / get_filename_input(
    current_month,
    current_year,
    current_month_text,
    prefix="",
    suffix="-actividad"
)

# Construct the full output file path
output_file = output_dir / get_filename_output(
    current_month,
    current_year,
    current_month_text,
    prefix="df-amex-tdc-",
    suffix=""
)

# Print the constructed file paths
print("Input file path: ", input_file)
print("Output file path: ", output_file)

In [ ]:
# Import monthly data from the specified Excel file

try:
    # Attempt to read the Excel file, skipping the first 6 rows
    df_amex = pd.read_excel(input_file, skiprows=6)
    print("Data imported successfully!")
except FileNotFoundError:
    # Handle the case where the input file is not found
    print(f"File not found: {input_file}")
except Exception as e:
    # Handle any other exceptions that may occur during file import
    print(f"An error occurred while reading the file: {e}")


### Data Description

Extract contains
- **Fecha**: Date in DD Month YYYY format
- **Fecha de Compra**: Purchase Date in DD Month YYYY format
- **Descripción**: Description that contains the entity.
- **Importe**: Charged amount in local currency (MXN)
- **Monto en moneda extranjera:** Charged amount in foreign currency
- **Tipo de cambio:** Rate
- **Información Adicional:** RFC (tax) information
- **Aparece en su Estado de Cuenta como:** Name used in the bank statement
- **Dirección:**Address
- **Población/Provincia:** Region
- **Código Postal:** Postal Code
- **País:** Country
- **Referencia:** Reference ID

In [ ]:
# Visualize the first few rows of the imported data
try:
    # Display the first five rows of the DataFrame
    print("Preview of the imported data:")
    display(df_amex.head())
except NameError:
    # Handle the case where the DataFrame does not exist
    print("The data has not been successfully loaded into a DataFrame. Please check the file import process.")
except Exception as e:
    # Handle any other unexpected errors
    print(f"An error occurred during data visualization: {e}")

### Basic Data Cleansing

This section focuses on preparing the financial dataset for analysis by removing irrelevant rows, handling missing values, transforming data types, and splitting columns for better structure. 

In [7]:
# Convert 'Fecha' column to datetime format (YYYY-MM-DD)
df_amex['Fecha'] = pd.to_datetime(df_amex['Fecha'], format='%d %b %Y')
# Converts the 'Fecha' column to a datetime object using the format "%d %b %Y".
# Example: "01 Jan 2024" becomes "2024-01-01".


# Convert the 'Importe' column to float
df_amex['Importe'] = df_amex['Importe'].astype(float)
# Ensures that the 'Importe' column contains numeric values in float format.
# This is important for numerical calculations or aggregations.

# Split the 'Monto en moneda extranjera' to obtain the currency and amount
df_amex[['Amount_Foreign_Currency', 'Foreign_Currency']] = df_amex['Monto en moneda extranjera'].str.extract(r'(\d+(?:\.\d+)?)\s*([A-Za-z]+)')

df_amex['Amount_Foreign_Currency'] = df_amex['Amount_Foreign_Currency'].astype(float)



In [8]:
def remove_accents(input_str):
    """
    Removes accents from a given string.

    Parameters:
    - input_str (str): The input string from which accents need to be removed.

    Returns:
    - str: The input string without accent marks.
    """
    # Step 1: Normalize the string to decompose accented characters into base characters and diacritical marks
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    
    # Step 2: Remove diacritical marks by filtering out combining characters
    return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

In [9]:
# Apply the function to clean the 'Descripción' column and create a new 'Desc_cleaned' column
df_amex['Desc_cleaned'] = df_amex['Descripción'].apply(remove_accents)

### Entity Harmonization

This section focuses on standardizing supplier names within the dataset by applying fuzzy matching techniques against a predefined mapping. The goal is to ensure consistency in supplier names for easier analysis and reporting.

In [10]:
def harmonize_supplier(supplier_name, entity_mapping):
    """
    Harmonizes a supplier name using fuzzy matching against a predefined mapping.

    Parameters:
    - supplier_name (str): The name of the supplier to be harmonized.
    - entity_mapping (dict): A dictionary where keys are potential supplier names 
                             and values are their harmonized names.

    Returns:
    - str: The harmonized supplier name if a close match is found; 
           otherwise, returns the original supplier name.
    """
    # Use fuzzy matching to find potential matches for the supplier name
    matches = get_close_matches(supplier_name, entity_mapping.keys(), n=1, cutoff=0.8)
    
    # Return the harmonized name if a close match is found
    if matches:
        return entity_mapping[matches[0]]
    
    # Return the original name if no close match is found
    return supplier_name

In [ ]:
# Extract the list of suppliers from the 'DESCRIPCION_ENTIDAD' column
suppliers_to_harmonize = df_amex['Desc_cleaned'].tolist()

# Step 1: Harmonize the supplier names using the mapping
harmonized_suppliers = [harmonize_supplier(supplier, entity_mapping) for supplier in suppliers_to_harmonize]

# Step 2: Add the harmonized supplier names back to the DataFrame as a new column
df_amex['Harmonized_Supplier'] = harmonized_suppliers


# Step 3: Display both 'DESCRIPCION_ENTIDAD' and 'Harmonized_Supplier' for comparison
print("Comparison of original and harmonized supplier names:")
display(df_amex[['Desc_cleaned', 'Harmonized_Supplier']])

# Step 4: Review the output and if necessary, update the entity_mapping.py dictionary and re-run the script

### Transaction Type Definition

In [12]:
# Assign transaction type descriptions based on the 'ABONO' column
df_amex['TXT_TRANSACTION_TYPE'] = np.where(
    df_amex['Importe'] > 0,  # Condition: If the value in 'ABONO' is not zero
    "Expense",             # Value to assign if the condition is True
    "Deposit"                 # Value to assign if the condition is False
)
# Transactions with higher than zero values in the 'Importe' column are categorized as "Expense".
# All other transactions are categorized as "Deposit".

In [ ]:
df_amex.head()

In [14]:
def harmonize_transaction_subtype(transaction_desc, transaction_subtype_mapping):
    """
    Harmonizes the transaction subtype description using fuzzy matching against a predefined mapping.

    Parameters:
    - transaction_desc (str): The name of the transaction descriptions to be harmonized.
    - entity_mapping (dict): A dictionary where keys are potential transaction descriptions 
                             and values are their harmonized names.

    Returns:
    - str: The harmonized transaction subtype if a close match is found; 
           otherwise, returns the original transaction description.
    """
    # Use fuzzy matching to find potential matches for the transaction subtype
    matches = get_close_matches(transaction_desc, transaction_subtype_mapping.keys(), n=1, cutoff=0.8)
    
    # Return the harmonized name if a close match is found
    if matches:
        return transaction_subtype_mapping[matches[0]]
    
    # Return the original name if no close match is found
    return transaction_desc

In [ ]:
# Extract the list of transaction descriptions from the 'Desc_cleaned' column
transactions_to_harmonize = df_amex['Desc_cleaned'].tolist()

# Step 1: Harmonize the transaction descriptions using the mapping
harmonized_transactions = [harmonize_transaction_subtype(transaction, transaction_subtype_mapping) for transaction in transactions_to_harmonize]

# Step 2: Add the harmonized supplier names back to the DataFrame as a new column
df_amex['Harmonized_Transaction_Subtype'] = harmonized_transactions

# Step 3: Display both 'DESCRIPCION_DETALLE' and 'harmonized_transactions' for comparison
print("Comparison of original and harmonized transaction:")
display(df_amex[['Desc_cleaned', 'Harmonized_Transaction_Subtype']])

# Step 4: Review the output and if necessary, update the transaction_subtype_mapping.p dictionary and re-run the script

In [ ]:
df_amex.head()

### Data Cleansing and Transformation

In [17]:
# Create a new column that identifies the source system
df_amex['KEY_SYSTEM'] = 'AMEX'

# Create a new column that identifies the source account
df_amex['KEY_ACCOUNT'] = 'AMEXTDCORO'
df_amex['TXT_ACCOUNT'] = 'AMEX TDC'

# Create a new column that combines CARGO and ABONO columns
df_amex['NUM_AMT_NET_REPORTING'] = df_amex['Importe'] * -1

# Create columns that contains the currency-related information
df_amex['NUM_AMT_DOCUMENT'] = np.where(
    pd.notna(df_amex['Amount_Foreign_Currency']),  # Check for non-NaN values
    df_amex['Amount_Foreign_Currency'],  # Use the amount if not NaN
    df_amex['Importe'] * -1  # Use this for NaN rows
)

df_amex['KEY_CURRENCY_DOCUMENT'] = np.where(
    pd.notna(df_amex['Amount_Foreign_Currency']),  # Check for non-NaN values
    'Foreign_currency',  # Use 'Foreign_currency' if not NaN
    'MXN'  # Use 'MXN' for NaN rows
)

df_amex['KEY_RATE'] = df_amex['Tipo de Cambio']

# Create new columns with time information
df_amex['KEY_MONTH'] = df_amex['Fecha'].dt.month
df_amex['KEY_YEAR'] = df_amex['Fecha'].dt.year

# Create a new column that contains the flag indicating whether the transaction is debit or credit
df_amex['FLG_DEBIT_CREDIT'] = np.where(df_amex['Importe'] > 0, 'C', 'D')

# Create new columns that identify the grouping operation (project, vacation, etc.). These will be blank and filled in later.
df_amex['KEY_OPERATION'] = ''
df_amex['TXT_OPERATION'] = ''

# Create new columns relevant for credit card transactions and payments in installments. Not relevant for this account.

# Logic to create DUE_DATE column
def calculate_due_date(row):
    if row['Importe'] >= 2400 and row['Amount_Foreign_Currency'] != 'USD':
        due_date = row['Fecha'] + relativedelta(months=2)
        return due_date.replace(day=3)
    else:
        return None  # Or any default value

df_amex['DUE_DATE'] = df_amex.apply(calculate_due_date, axis=1)

df_amex['KEY_PAYMENT_TERM'] = 'MSI3'
df_amex['TXT_PAYMENT_TERM'] = '3 MSI'

# Logic to calculate the amount due
def calculate_due_amount(row):
    if row['Desc_cleaned'] == "MESES EN AUTOMATICO NACIONAL":
        return row['Importe'] * 3 * -1
    else:
        return None  # Or any default value


df_amex['NUM_AMT_DUE'] = df_amex.apply(calculate_due_amount, axis=1).fillna(0)

df_amex['KEY_ID_DUE'] = ''
df_amex['TXT_ENTITY_DUE'] = ''
df_amex['TXT_DESC_DUE'] = ''

# Create new columns with the country information. This account does not provide the country information.

# Mapping of country names to country codes
country_mapping = {
    "MEXICO": "MX",
    "UNITED STATES": "US",
    "GBP": "UK",
    "EUR": "EU"
}

df_amex['KEY_COUNTRY'] = df_amex['País'].map(country_mapping).fillna("MX")

# Mapping of country names to country names
name_country_mapping = {
    "MEXICO": "Mexico",
    "UNITED STATES": "United States",
    "GBP": "United Kingdom"
}

df_amex['TXT_COUNTRY'] = df_amex['País'].map(name_country_mapping).fillna("Mexico")

# Create a new column with the purchase document number, blank initially.
df_amex['KEY_PURCH_DOC_NO'] = ''

# Create a new column with internal flag
df_amex['FLG_INTERNAL'] = np.where(df_amex['Harmonized_Supplier'] == 'BBVA Ricardo Perez Castillo', 'Y', 'N')

# Create a new column with flag indicating whether the transaction was canceled
df_amex['FLG_CANCEL'] = ''

# Create a new column with flag indicating whether the transaction was refunded
df_amex['FLG_REFUND'] = (
    (df_amex['Importe'] < 0) & 
    (~df_amex['Desc_cleaned'].isin(["MONTO A DIFERIR MESES EN AUTOMATICO", "GRACIAS POR SU PAGO CON CARGO A BBVA"]))
)

# Create new columns that will be filled later with the master tables
df_amex['KEY_ENTITY'] = ''
df_amex['KEY_TRANSACTION_TYPE'] = ''
df_amex['KEY_TRANSACTION_SUBTYPE'] = ''

# Rename the columns to match the standard naming convention
df_amex.rename(columns={
    'Fecha': 'KEY_DATE',
    'Desc_cleaned': 'TXT_DESC',
    'Harmonized_Supplier': 'TXT_ENTITY',
    'Harmonized_Transaction_Subtype': 'TXT_TRANSACTION_SUBTYPE',
    'Referencia': 'KEY_ID'
}, inplace=True)

In [18]:
# Order the columns based on the standard order
df_amex = df_amex[[
    'KEY_ID', 'KEY_SYSTEM', 'KEY_ACCOUNT', 'TXT_ACCOUNT', 'KEY_DATE', 'KEY_MONTH', 'KEY_YEAR',
    'KEY_ENTITY', 'TXT_ENTITY', 'KEY_TRANSACTION_TYPE', 'TXT_TRANSACTION_TYPE', 'KEY_TRANSACTION_SUBTYPE',
    'TXT_TRANSACTION_SUBTYPE', 'TXT_DESC', 'NUM_AMT_NET_REPORTING', 'NUM_AMT_DOCUMENT', 'KEY_CURRENCY_DOCUMENT',
    'KEY_RATE', 'FLG_DEBIT_CREDIT', 'KEY_OPERATION', 'TXT_OPERATION', 'DUE_DATE', 'KEY_PAYMENT_TERM',
    'TXT_PAYMENT_TERM', 'NUM_AMT_DUE', 'KEY_ID_DUE', 'TXT_ENTITY_DUE', 'TXT_DESC_DUE', 'KEY_COUNTRY',
    'TXT_COUNTRY', 'KEY_PURCH_DOC_NO', 'FLG_INTERNAL', 'FLG_CANCEL', 'FLG_REFUND'
]]

In [ ]:
# Final visualization of the processed data
df_amex.head()

## 05. Exporting

In [ ]:
# Export the cleaned and processed DataFrame to a CSV file

try:
    # Export DataFrame to the specified output file
    df_bbva.to_csv(output_file, index=False)
    print(f"File successfully exported to: {output_file}")
except FileNotFoundError:
    # Handle the case where the output directory does not exist
    print(f"Output path not found: {output_file}")
except PermissionError:
    # Handle permission issues when writing to the file
    print(f"Permission denied while trying to write to: {output_file}")
except Exception as e:
    # Handle any other unforeseen errors during the export
    print(f"An unexpected error occurred while writing the file: {e}")